# Data Collection

### Charts

- [acharts.co](https://acharts.co/) (different countries, per week)
- [swisscharts.com](http://swisscharts.com/charts/singles/) (swiss charts, per week)
- [ct30.com](http://www.ct30.com/) (community created, end-year, international top-100)
- [Wikipedia](https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2020) (archived data from billboard.com)
- [Generasia](https://www.generasia.com/wiki/Category:Rankings) (open asian charts)
- [mediatraffic.de](http://www.mediatraffic.de/previous.htm) (per week / end-year international charts)

In [5]:
import pandas as pd

df = pd.read_csv('../data/billboard_2010_2024.csv')
df.head()

,place,title,artist,year
0,1,Tik Tok,Kesha,2010
1,2,Need You Now,Lady Antebellum,2010
2,3,"Hey, Soul Sister",Train,2010
3,4,California Gurls,Katy Perry,2010
4,5,OMG,Usher,2010


#### Wikipedia

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [54]:
url = "https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_{year}"

items_per_year = dict()

for year in range(2010, 2025):
    res = requests.get(url.format(year=year))
    soup = BeautifulSoup(res.text, "html.parser")

    items_raw = soup.find("table", {"class": "wikitable"}).tbody.find_all("tr")[1:]
    
    items = []
    
    for x in items_raw:
        parts = x.find_all("td")
        if len(parts) == 3:
            place = parts[0].text
            title = parts[1].a.text
            artist = parts[2].a.text
        else:
            place = parts[0].text
            title = parts[1].a.text
            
        items.append({"place": int(place.strip()), "title": title, "artist": artist})
    
    items_per_year[year] = items

In [55]:
df_general = pd.DataFrame()

# Create a DataFrame from the dictionary
for year, items in items_per_year.items():
    df_year = pd.DataFrame(items)

    # Add the year column
    df_year['year'] = year

    # Append the new DataFrame to the general DataFrame
    df_general = pd.concat([df_general, df_year], ignore_index=True)

df_general

,place,title,artist,year
0,1,Tik Tok,Kesha,2010
1,2,Need You Now,Lady Antebellum,2010
2,3,"Hey, Soul Sister",Train,2010
3,4,California Gurls,Katy Perry,2010
4,5,OMG,Usher,2010
...,...,...,...,...
1495,96,Bulletproof,Nate Smith,2024
1496,97,Fe!n,Travis Scott,2024
1497,98,The Painter,Cody Johnson,2024
1498,99,Down Bad,Taylor Swift,2024


In [56]:
df_general.to_csv("../data/billboard_2010_2024.csv", index=False)

### Lyrics

- [AZlyrics](https://www.azlyrics.com/)
- [LyricsFind](https://lyrics.lyricfind.com/)
- [Genius API](https://docs.genius.com/)
- [Lyrics.com API](https://www.lyrics.com/lyrics_api.php)
- [MLDB.org](https://www.mldb.org/) (limited data)

#### AZlyrics

In [43]:
!pip install azapi

In [ ]:
import azapi

API = azapi.AZlyrics()

Google found nothing!



In [69]:
for song in items_per_year[2013]:
    API.artist = song["artist"]
    API.title = song["title"]
    
    API.getLyrics()

    print(API.lyrics)

KeyboardInterrupt: 

#### Genius

In [8]:
!pip install git+https://github.com/johnwmillr/LyricsGenius.git

  Cloning https://github.com/johnwmillr/LyricsGenius.git to /private/var/folders/db/z1j0x3f51c56c00nhhmhl7qw0000gn/T/pip-req-build-nextf_jh
  Running command git clone --filter=blob:none --quiet https://github.com/johnwmillr/LyricsGenius.git /private/var/folders/db/z1j0x3f51c56c00nhhmhl7qw0000gn/T/pip-req-build-nextf_jh
  Resolved https://github.com/johnwmillr/LyricsGenius.git to commit 7635cff7af35bfa5b6e8ec36ca6921faf7014cc3
  Preparing metadata (setup.py) ... done
  Created wheel for lyricsgenius: filename=lyricsgenius-3.2.0-py3-none-any.whl size=45052 sha256=c123ff1f05c80178b70977246eee50f74c3f3d4bb68007f7ffc5d0ea9ea53e72
  Stored in directory: /private/var/folders/db/z1j0x3f51c56c00nhhmhl7qw0000gn/T/pip-ephem-wheel-cache-2bn57vvd/wheels/a2/6d/80/1297c611c6dc8bd51f83121a1e209257591bc9a0486bd2e279
Successfully built lyricsgenius


In [ ]:
!pip install tqdm

In [1]:
import lyricsgenius
import os
from tqdm.notebook import tqdm

genius = lyricsgenius.Genius()

In [2]:
song = genius.search_song("Thrift Shop", "Macklemore & Ryan Lewis")
song.lyrics

Searching for "Thrift Shop" by Macklemore & Ryan Lewis...
Done.


'[Intro]\nHey, Macklemore, can we go thrift shopping?\nWhat-what, what, what?\nWhat-what, what, what?\nWhat-what, what, what?\nWhat-what, what, what? (Da)\nWhat-what, what, what? (Da-dum, ba-dum, ba-da-do-da)\nWhat-what, what, what? (Da-dum, ba-dum, ba-da-do-da)\nWhat-what, what, what? (Da-dum, ba-dum, ba-da-do-da)\nWhat-what, what, what? (Da-dum, ba-dum, ba-da-do-da)\nDa-dum, ba-dum, ba-da-do-da\nDa-dum, ba-dum, ba-da-do-da\nDa-dum, ba-dum, ba-da-do-da\nDa-dum, ba-dum, ba-da-do-da\n[Chorus: Wanz & Macklemore]\nI\'m gonna pop some tags\nOnly got twenty dollars in my pocket\nI\'m, I\'m, I\'m huntin\', lookin\' for a come up\nThis is fucking awesome (Now)\n\n[Verse 1: Macklemore]\nWalk into the club like, "What up? I got a big cock"\nNah, I\'m just pumped, I bought some shit from a thrift shop\nIce on the fringe is so damn frosty\nThe people like, "Damn, that\'s a cold-ass honkey"\nRollin\' in hella deep, headed to the mezzanine\nDressed in all pink \'cept my gator shoes, those are green

In [4]:
# process None values
song = genius.search_song("Can't Hold Us", "Macklemore & Ryan Lewis")
song.lyrics

Searching for "Can't Hold Us" by Macklemore & Ryan Lewis...
Specified song does not contain lyrics. Rejecting.


AttributeError: 'NoneType' object has no attribute 'lyrics'

In [6]:
from IPython.display import clear_output

lyrics_per_year = dict()
processed_years = set([2013]    )

for year in tqdm(range(2010, 2025), "Years"):
    if year in processed_years:
        continue

    lyrics = []
    
    clear_output()
    
    for song in tqdm(df[df.year == year].to_dict(orient="records"), f"Songs, year {year}"):
        try:
            song_genius = genius.search_song(song["title"], song["artist"])
            lyrics.append(song_genius.lyrics)
            
            if not os.path.exists(f"../data/lyrics/{song['title']}_{song['artist']}.txt"):
                with open(f"../data/lyrics/{song['title']}_{song['artist']}.txt", "w") as f:
                    f.write(song_genius.lyrics)
        except:
            print(f"Error with {song['title']} by {song['artist']}")
            lyrics.append(None)
            
    lyrics_per_year[year] = lyrics

Songs, year 2024:   0%|          | 0/100 [00:00<?, ?it/s]

Searching for "Lose Control" by Teddy Swims...
Done.
Searching for "A Bar Song (Tipsy)" by Shaboozey...
Done.
Searching for "Beautiful Things" by Benson Boone...
Done.
Searching for "I Had Some Help" by Post Malone...
Done.
Searching for "Lovin on Me" by Jack Harlow...
Done.
Searching for "Not Like Us" by Kendrick Lamar...
Done.
Searching for "Espresso" by Sabrina Carpenter...
Done.
Searching for "Million Dollar Baby" by Tommy Richman...
Done.
Searching for "I Remember Everything" by Zach Bryan...
Done.
Searching for "Too Sweet" by Hozier...
Done.
Searching for "Stick Season" by Noah Kahan...
Done.
Searching for "Cruel Summer" by Taylor Swift...
Done.
Searching for "Greedy" by Tate McRae...
Done.
Searching for "Like That" by Future...
Done.
Searching for "Birds of a Feather" by Billie Eilish...
Done.
Searching for "Please Please Please" by Sabrina Carpenter...
Done.
Searching for "Agora Hills" by Doja Cat...
Done.
Searching for "Good Luck, Babe!" by Chappell Roan...
Done.
Searching for

In [ ]:
for year, lyrics in lyrics_per_year.items():
    df.loc[df.year == year, "lyrics"] = lyrics

In [ ]:
df.lyrics.isna().sum()

In [37]:
# df.loc[df.year == 2013, "lyrics"] = lyrics
df.to_csv("../data/billboard_2010_2024.csv", index=False)

,place,title,artist,year,lyrics
300,1,Thrift Shop,Macklemore & Ryan Lewis,2013,"[Intro]\nHey, Macklemore, can we go thrift sho..."
301,2,Blurred Lines,Robin Thicke,2013,[Intro: Pharrell Williams & Robin Thicke]\nEve...
302,3,Radioactive,Imagine Dragons,2013,None
303,4,Harlem Shake,Baauer,2013,"[Pre-Chorus]\nCon los terroristas, -tas, -tas,..."
304,5,Can't Hold Us,Macklemore & Ryan Lewis,2013,None
...,...,...,...,...,...
395,96,Brave,Sara Bareilles,2013,[Verse 1]\nYou can be amazing\nYou can turn a ...
396,97,Let Her Go,Passenger,2013,[Produced by Mike Rosenberg and Chris Vallejo]...
397,98,Runnin' Outta Moonlight,Randy Houser,2013,[Verse 1]\nDon't you worry 'bout gettin' fixed...
398,99,I'm Different,2 Chainz,2013,[Intro]\nYeah\nYeah\n2 Chainz\nMustard on the ...


### Descriptions

We suggest to limit the possible labels to modes of transport (according to [Wikipedia](https://en.wikipedia.org/wiki/Mode_of_transport)): **_human-powered_**, **_animal-powered_**, **_railways_**, **_roadways_**, **_water transport_**, **_air transport_**.

In [45]:
definitions_dict = {
    "mode of transport": 
    "A mode of transport is a method or way of travelling, or of transporting people or cargo.[1] The different modes of transport include air, water, and land transport, which includes rails or railways, road and off-road transport. Other modes of transport also exist, including pipelines, cable transport, and space transport. Human-powered transport and animal-powered transport are sometimes regarded as distinct modes, but they may lie in other categories such as land or water transport."
    "In general, transportation refers to the moving of people, animals, and other goods from one place to another, and means of transport refers to the transport facilities used to carry people or cargo according to the chosen mode. Examples of the means of transport include automobile, airplane, ship, truck, and train. Each mode of transport has a fundamentally different set of technological solutions. Each mode has its own infrastructure, vehicles, transport operators and operations.",
    "human-powered": "Human powered transport, a form of sustainable transportation, is the transport of people and/or goods using human muscle-power, in the form of walking, running and swimming. Modern technology has allowed machines to enhance human power. Human-powered transport remains popular for reasons of cost-saving, leisure, physical exercise, and environmentalism; it is sometimes the only type available, especially in underdeveloped or inaccessible regions."
    "Although humans are able to walk without infrastructure, the transport can be enhanced through the use of roads, especially when using the human power with vehicles, such as bicycles and inline skates. Human-powered vehicles have also been developed for difficult environments, such as snow and water, by watercraft rowing and skiing; even the air can be entered with human-powered aircraft.", 
    "animal-powered": "Animal-powered transport is the use of working animals for the transport of people and/or goods. Humans may use some of the animals directly, use them as pack animals for carrying goods, or harness them, alone or in teams, to pull watercraft, sleds, or wheeled vehicles."
    "Riding animals are animals that people use as mounts in order to perform tasks such as traversing across long distances or over rugged terrain, hunting on horseback or with some other riding animal, patrolling around rural and/or wilderness areas, rounding up and/or herding livestock or even for recreational enjoyment. They mainly include equines such as horses, donkeys, and mules; bovines such as cattle, water buffalo, and yak. In some places, elephants, llamas and camels are also used. Dromedary camels are in arid areas of Australia, North Africa and the Middle East; the less common Bactrian camel inhabits central and East Asia; both are used as working animals. On occasion, reindeer, though usually driven, may be ridden. Certain wild animals have been tamed and used for riding, usually for novelty purposes, including the zebra and the ostrich. Some mythical creatures are believed to act as divine mounts, such as garuda in Hinduism (See vahana for divine mounts in Hinduism) and the winged horse Pegasus in Greek mythology.", 
    "railways": "Rail transport is a means of conveyance of passengers and goods by way of wheeled vehicles running on rail track, known as a railway or railroad. The rails are anchored perpendicular to railroad train consists of one or more connected vehicles that run on the rails. Propulsion is commonly provided by a locomotive, that hauls a series of unpowered cars, that can carry passengers or freight. The locomotive can be powered by steam, diesel or by electricity supplied by trackside systems. Alternatively, some or all the cars can be powered, known as a multiple unit. Also, a train can be powered by horses, cables, gravity, pneumatics and gas turbines. Railed vehicles move with much less friction than rubber tires on paved roads, making trains more energy efficient, though not as efficient as ships."
    "Intercity trains are long-haul services connecting cities;[12] modern high-speed rail is capable of speeds up to 430 km/h (270 mph), but this requires a specially built track. Regional and commuter trains feed cities from suburbs and surrounding areas, while intra-urban transport is performed by high-capacity tramways and rapid transits, often making up the backbone of a city's public transport. Freight trains traditionally used box cars, requiring manual loading and unloading of the cargo. Since the 1960s, container trains have become the dominant solution for general freight, while large quantities of bulk are transported by dedicated trains.", 
    "roadways": "A road is an identifiable route of travel, usually surfaced with gravel, asphalt or concrete, and supporting land passage by foot or by a number of vehicles."
    "The most common road vehicle in the developed world is the automobile, a wheeled passenger vehicle that carries its own motor. As of 2002, there were 591 million automobiles worldwide.[citation needed] Other users of roads include motorcycles, buses, trucks, bicycles and pedestrians, and special provisions are sometimes made for each of these. For example, bus lanes give priority for public transport, and cycle lanes provide special areas of road for bicycles to use."
    "Automobiles offer high flexibility, but are deemed with high energy and area use, and the main source of noise and air pollution in cities; buses allow for more efficient travel at the cost of reduced flexibility.[13] Road transport by truck is often the initial and final stage of freight transport.", 
    "water transport": "Water transport is the process of transport that a watercraft, such as a bart, ship or sailboat, makes over a body of water, such as a sea, ocean, lake, canal, or river. If a boat or other vessel can successfully pass through a waterway it is known as a navigable waterway. The need for buoyancy unites watercraft, and makes the hull a dominant aspect of its construction, maintenance and appearance. When a boat is floating on the water the hull of the boat is pushing aside water where the hull now is, this is known as displacement."
    "In the 1800s, the first steamboats were developed, using a steam engine to drive a paddle wheel or propeller to move the ship. The steam was produced using wood or coal. Now, most ships have an engine using a slightly refined type of petroleum called bunker fuel. Some ships, such as submarines, use nuclear power to produce the steam. Recreational or educational craft still use wind power, while some smaller craft use internal combustion engines to drive one or more propellers, or in the case of jet boats, an inboard water jet. In shallow draft areas, hovercraft are propelled by large pusher-prop fans."
    "Although slow, modern sea transport is a highly effective method of transporting large quantities of non-perishable goods. Commercial vessels, nearly 35,000 in number, carried 7.4 billion tons of cargo in 2007.[14] Transport by water is significantly less costly than air transport for transcontinental shipping;[15] short sea shipping and ferries remain viable in coastal areas.[16][17]", 
    "air transport": "A fixed-wing aircraft, typically airplane, is a heavier-than-air flying vehicle, in which the special geometry of the wings generates lift and then lifts the whole vehicle. Fixed-wing aircraft range from small trainers and recreational aircraft to large airliners and military cargo aircraft. For short distances or in places without runways, helicopters can be operable.[2] (Other types of aircraft, like autogyros and airships, are not a significant portion of air transport.)"
    "Air transport is one of the fastest method of transport, Commercial jets reach speeds of up to 955 kilometres per hour (593 mph) and a considerably higher ground speed if there is a jet stream tailwind, while piston-powered general aviation aircraft may reach up to 555 kilometres per hour (345 mph) or more. This celerity comes with higher cost and energy use,[3] and aviation's impacts to the environment and particularly the global climate require consideration when comparing modes of transportation.[4] The Intergovernmental Panel on Climate Change (IPCC) estimates a commercial jet's flight to have some 2-4 times the effect on the climate than if the same CO2 emissions were made at ground level, because of different atmospheric chemistry and radiative forcing effects at the higher altitude.[5] U.S. airlines alone burned about 16.2 billion gallons of fuel during the twelve months between October 2013 and September 2014.[6] WHO estimates that globally as many as 500,000 people at a time are on planes.[3] The global trend has been for increasing numbers of people to travel by air, and individually to do so with increasing frequency and over longer distances, a dilemma that has the attention of climate scientists and other researchers,[7][8][9] along with the press.[10][11] The issue of impacts from frequent travel, particularly by air because of the long distances that are easily covered in one or a few days, is called hypermobility and has been a topic of research and governmental concern for many years." 
}

In [46]:
import json

with open("../data/definitions.json", "w") as f:
    json.dump(definitions_dict, f, indent=4)